# Selenium으로 네이버 뉴스 스크랩
- 검색필터: 관련도 순

## version 1. Selenium으로 직접 조작

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# Chrome WebDriver 설정
driver = webdriver.Chrome(ChromeDriverManager().install(), options=webdriver.ChromeOptions())

driver.get('https://www.naver.com')

ele = driver.find_element(By.ID, 'query')
ele.send_keys('광진구')
ele.send_keys(Keys.ENTER)

news = driver.find_element(by=By.LINK_TEXT, value='뉴스')
news.click()

# 날짜 필터 걸기
driver.find_element(by=By.LINK_TEXT, value='옵션').click()
driver.find_element(by=By.LINK_TEXT, value='직접입력').click()
## selector scroll 

## version 2. url 값 변경

In [56]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import Select

import pandas as pd 

def main():
    # 검색필터 설정
    keyword = input('검색어를 입력해주세요.')
    start_date = input('시작 날짜를 입력해주세요(예: 2023.01.01)')
    end_date = input('마지막 날짜를 입력해주세요(예: 2023.12.31)')
    
    # url 접속
    url = f'https://search.naver.com/search.naver?where=news&query={keyword}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20230101to20240319&is_sug_officeid=0&office_category=0&service_area=0'
    driver.get(url)

    # 스크롤
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        time.sleep(1)
        new_height = driver.execute_script('return document.body.scrollHeight')
    
        if last_height != new_height:
            last_height = new_height
        else:
            break

    # 기사제목 & url 가져오기 
    titles = []
    urls = []
    pub_dates = []
    press = []

    bs = BeautifulSoup(driver.page_source, 'lxml')
    articles = bs.findAll('div', attrs={'class':'news_area'})
    for article in articles:
        titles.append(article.find('a', attrs={'class':'news_tit'}).text)
        urls.append(article.find('a', attrs={'class':'news_tit'})['href'])
        pub_dates.append(article.find('span', attrs={'class':'info'}).text)
        press.append(article.find('a', attrs={'class':'info press'}).text)

    contents = []
    comments = []
    # # 기사내용 & 댓글 가져오기 
    for url in urls:
        time.sleep(3)
        driver.get(url)
        bs = BeautifulSoup(driver.page_source, 'lxml')
        try:
            contents.append(bs.find('div', attrs={'itemprop':'articleBody'}).text)
        except:
            contents.append('')
            

    # cvs로 반환
    df = pd.DataFrame({'press':press, 'title':titles, 'pub_date':pub_dates, 'url':urls, 'contents':contents})
    df.to_csv(f'{keyword}_news_{start_date}_{end_date}.csv', index=False, encoding='utf-8-sig')

In [58]:
main()

검색어를 입력해주세요. 광진구
시작 날짜를 입력해주세요(예: 2023.01.01) 2023.02.01
마지막 날짜를 입력해주세요(예: 2023.12.31) 2023.02.28


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=60553): Max retries exceeded with url: /session/1eb62886585329aeddec9d010ca1ad44/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x123351210>: Failed to establish a new connection: [Errno 61] Connection refused'))